## Data Loading
Loads in the big data matrix and the stfp matrix created by matlab

In [89]:
from scipy.io import loadmat

def load_in_data():
    file = scipy.io.loadmat("data/simulationData.mat")
    data = file["data"]
    return data

def load_stfp():
    file = scipy.io.loadmat("data/stfp.mat")
    stfp = file["stfp"]
    return stfp


## Data processing
Processes the data just as in the matlab version

In [90]:
# Set the notebook directory as the working directory
%pwd
%cd C:\Users\Rafael\Documents\GitHub\BeyondHulten

[Errno 2] No such file or directory: 'C:UsersRafaelDocumentsGitHubBeyondHulten'
/home/franzs/Schreibtisch/BeyondHulten


In [91]:
import numpy as np
import pandas as pd
import scipy

#data is the IO matrix for 88 sectors over 46 years (1960-2005)
data2 = load_in_data()
ind = pd.MultiIndex.from_arrays(data2[:, (0,1)].T, names=('year','sec'))
data = pd.DataFrame(data2[:, 2:93], index=ind)
del data2, ind

In [92]:
#Column 0 of datavector contains gross output for each sector
print(data.loc[(1960,2),0]*2) #first column times two
print(data.loc[(1960,2)].sum()) #summed across all columns (including the sum in column 0 already)
#Column 1 and 2 contain labour and capital
print(data.loc[(2005,2),0:2])
#Columns 3 to 90 contain the 88 sectors
#Colum 91 contains noncompetitive imports


3434.026
3433.7749999999996
0    88366.023
1    25623.402
2    31151.107
Name: (2005.0, 2.0), dtype: float64


In [93]:
#delete 10 sectors
removable_sectors = [59] + list(range(79, 88))
data2 = data.drop(index=removable_sectors, level=1)

#key aggregates
grossy = data2.loc[:,0].unstack().T
capital = data2.loc[:,1].unstack().T
labor = data2.loc[:,2].unstack().T
vadd = labor + capital

#remaining sector list
sec = list(range(1,89))
for i in removable_sectors:
    sec.remove(i)
len(sec)

#delete sectors which don't produce anything
grossy.loc[(grossy==0).all(axis=1)] #sectors 8 and 62
zeroprod = list(grossy.loc[(grossy==0).all(axis=1)].index.values)

grossy = grossy.drop(index=zeroprod)
capital = capital.drop(index=zeroprod)
labor = labor.drop(index=zeroprod)
vadd = vadd.drop(index=zeroprod)

stfp = load_stfp()
Σ = np.cov(stfp, rowvar=True)
μ = np.mean(stfp, axis=1)

## Creating usable variables
Function that creates the metrics for a given year

In [217]:
def get_variables(year):
    IO = data.loc[year,:]
    
    sectors_to_remove = list(map(int, grossy[year].index))
    temp = list(map(lambda x: x - 1 ,sectors_to_remove))
    IO = IO.loc[sectors_to_remove,temp]

    Ω = IO.div(IO.sum(axis = 1),axis = 0).to_numpy()
    
    α = vadd[year]/grossy[year]
    β = (np.eye(len(grossy)) - np.diag(1 - α) @ Ω).T @ grossy[year]
    β[β < 0] = 0
    β = β / np.sum(β)
    λ = np.linalg.inv( np.eye(len(grossy)) - np.diag(1 - α) @ Ω).T @ β
    L = λ * α

    return α, β, Ω, L, λ,IO

## Elasticities

In [218]:
ε = .5;
θ = 0.001;
σ = .9;

# Construction Site Below HERE

## Objective Function

Here the objective function in `SimulationDerivs.m` is formulated, also the Jacobian is given, to helpt the solver.

In [219]:
def problem(X, A, β, Ω, α, ε, θ, σ, L):
    N = len(α)
    p = X[:N]
    y = X[N:]

    Out = np.zeros(2 * N, dtype=X.dtype)

    q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
    w = p * (A ** ((ε - 1) / ε)) * (α ** (1 / ε)) * (y ** (1 / ε)) * L ** (-1 / ε)
    C = np.dot(w, L)

    Out[:N] = p - (A ** (ε - 1) * (α * w ** (1 - ε) + (1 - α) * q ** (1 - ε))) ** (1 / (1 - ε))
    Out[N:] = y - np.dot(np.dot(np.dot(np.dot(np.dot(y.T, np.diag(p) ** ε), np.diag(A) ** (ε - 1)), np.diag(q) ** (θ - ε)), np.diag(1 - α)), np.dot(np.dot(Ω, np.diag(p) ** (-θ)), β)) - np.dot(β.T, np.diag(p) ** (-σ) * C)

    return Out


## Generating Random Shocks

In [220]:
from scipy.stats import multivariate_normal

def generate_random_shock(Σ, α, Ω, λ):
    n = len(Σ)
    cov_matrix = np.diag(np.diag(Σ))
    A = np.exp(multivariate_normal.rvs(mean=-0.5 * np.diag(Σ), cov=cov_matrix))
    
    inv_term = np.linalg.inv(np.eye(n) - np.diag(1 - α) @ Ω)
    log_A = np.log(A)
    init = np.concatenate((np.exp(-inv_term @ log_A), λ / np.exp(-inv_term @ log_A)))
    
    return A, init


## Solving for shocks

In [224]:
trials = 100
GDP = np.zeros(trials)
λ_sim = np.zeros((76, trials))
α, β, Ω, L, λ,IO = get_variables(1973)
λ

array([0.56797846, 0.0984713 , 0.14119061, 0.07622551, 0.01154578,
       0.00483282, 0.01873056, 0.10694405, 0.01611619, 0.0073599 ,
       0.01457226, 0.03907975, 0.03115455, 0.03595211, 0.01320776,
       0.00947077, 0.00770112, 0.01309429, 0.00505481, 0.0064299 ,
       0.03901657, 0.01316918, 0.0035707 , 0.00421526, 0.00522953,
       0.00275391, 0.00392635, 0.00707333, 0.06694498, 0.0025908 ,
       0.01647085, 0.03163935, 0.00313416, 0.01953194, 0.01058995,
       0.00807804, 0.03170609, 0.00444098, 0.01789236, 0.02130856,
       0.00994803, 0.00745945, 0.02617682, 0.00694997, 0.0078579 ,
       0.008782  , 0.02280724, 0.00323961, 0.0193152 , 0.01053034,
       0.00150734, 0.0680947 , 0.05906693, 0.01558726, 0.03295491,
       0.01558063, 0.00843816, 0.02277378, 0.06308037, 0.00792633,
       0.01755452, 0.02070709, 0.00231084, 0.0087329 , 0.00778167,
       0.00284444, 0.00690486, 0.01540047, 0.0027955 , 0.0128924 ,
       0.00360711, 0.00914225, 0.0080856 , 0.01343573, 0.00341